Code to run the rudimentary design(without CNN)

In [27]:
# Import all necessary files

# import numpy as np
# import torch
# import librosa
# import cv2
import os
import moviepy.editor as mp
from Components.Phoneme_Extraction import speech2text
from Components.p2fa_py3.p2fa import align
from Components.Viseme_Extraction import viseme
from Components.Mouth_Labelling import mouthLabelling
from Components.Mismatch_Estimator import mismatch

In [3]:
                                            # Extract the audio from the input video file

# Insert Local Video File Path

video_path = "./Datasets/Video/obama1.mp4"
clip = mp.VideoFileClip(video_path)
    
# Insert Local Audio File Path
audio_path = video_path.replace('.mp4', '.wav').replace('Video', 'Audio')
print(audio_path)
clip.audio.write_audiofile(audio_path)

./Datasets/Audio/zuckerbergDeepfake.wav
MoviePy - Writing audio in ./Datasets/Audio/zuckerbergDeepfake.wav


MoviePy - Done.


In [4]:
# Converting Audio file to Text

transcription = speech2text.speech_transcription(audio_path)
print(transcription)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IMAGINE THIS FOR A SECOND ONE MAN WITH TOTAL CONTROL OF BILLIONS OF PEOPLE STOLEN DATA ALL THEIR SECRETS THEIR LIVES THEIR FUTURES I OWE IT ALL TO SPECTRE SPECTRES SHOWED ME THAT WHOEVER CONTROLS THE DATA CONTROLS THE FUTURE AN


In [14]:
# Write the variable value to the text file
text_path = video_path.replace('.mp4', '.txt').replace('Video', 'Text')

with open(text_path, 'w') as file:
    file.write(transcription.replace(' ', '\n'))

In [21]:
# Phoneme Extraction

# Perform alignment using P2FA
phoneme_timestamps, word_timestamps, pff = align.align(audio_path, text_path)

# Extract the timestamps of a specific phoneme
target_phonemes = ["M", "B", "P"] # Closed mouth phonemes that we want to extract
timestamps = []
for phoneme in target_phonemes:
    target_phoneme_timestamps = [(start, end) for p, start, end in phoneme_timestamps if p == phoneme]
    timestamps.extend(target_phoneme_timestamps)

# Print the timestamps of the target phoneme
print(f"Timestamps of Closed Mouth: {timestamps}")

Resampling wav file from 44100 to 11025...
SKIPPING WORD SPECTRES
/tmp/p2fa/sound.wav -> /tmp/p2fa/tmp.plp 
Timestamps of Closed Mouth: [(0.41156462585034015, 0.4913832199546485), (2.576643990929705, 2.6365079365079365), (13.581632653061224, 13.621541950113379), (4.462358276643991, 4.512244897959183), (5.060997732426303, 5.090929705215419), (5.220634920634921, 5.250566893424036), (12.833333333333332, 12.873242630385487)]


In [37]:
# Viseme Extraction

#import libraires
import cv2
import numpy as np
frames=viseme.extract_frames(video_path, timestamps)

# Define the output folder
frame_path = "Datasets/Viseme_frames/"

# Loop through the frames and save them to the output folder
for i, (frame,index,frame_seqnum) in enumerate(frames):
    filename = "Frame_{}_{}.jpg".format(index,frame_seqnum)

    cv2.imwrite( frame_path + filename, frame)


Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Fra

[mov,mp4,m4a,3gp,3g2,mj2 @ 0xb6ab140] moov atom not found


In [33]:
# MAR Calculation

mar = mouthLabelling.mouth_aspect_ratio(frame_path)
print("Mouth aspect ratio: ", len(mar), " items")
print(len(mar))
for i in range(len(mar)):
  print (mar[i])

/home/reubensuju/Reuben/NITC_C-GCC_Repo/Sem8/Project_Part_2/deepfake-detection
[INFO] loading facial landmark predictor...
Mouth aspect ratio:  42  items
['Frame_6_2.jpg', 0.5767972674053732]
['Frame_1_1.jpg', 0.7174527310059746]
['Frame_3_5.jpg', 0.5915625522877966]
['Frame_0_2.jpg', 0.5221483679937239]
['Frame_3_4.jpg', 0.5915625522877966]
['Frame_3_2.jpg', 0.5586043624760154]
['Frame_0_1.jpg', 0.5096097386017643]
['Frame_1_2.jpg', 0.7398219124831164]
['Frame_2_5.jpg', 0.5863194699698894]
['Frame_1_0.jpg', 0.7174527310059746]
['Frame_3_0.jpg', 0.5249636615625813]
['Frame_1_5.jpg', 0.7498903829190143]
['Frame_4_0.jpg', 0.5418434750578494]
['Frame_4_2.jpg', 0.5606139519520328]
['Frame_2_1.jpg', 0.5504281305003996]
['Frame_2_4.jpg', 0.5863194699698894]
['Frame_0_3.jpg', 0.5221483679937239]
['Frame_4_4.jpg', 0.5606139519520328]
['Frame_6_5.jpg', 0.6658602737399514]
['Frame_5_4.jpg', 0.4887308375627064]
['Frame_5_2.jpg', 0.4821862716710384]
['Frame_0_0.jpg', 0.5096097386017643]
['Frame_1_

In [48]:
def extract_num(frame):
    return int(frame[0].split("_")[1]), int(frame[0].split("_")[2].split(".")[0])

# Sort frames based on numerical value in filename
mar.sort(key=lambda frame: (extract_num(frame)[0], extract_num(frame)[1]))

num_rows = len(mar) // 6  # Compute number of rows required
# Initialize 2D array with zeros
mar_values = np.zeros((num_rows, 6))

# Loop through sorted frames list and populate 2D array
for i, frame in enumerate(mar):
    row = i // 6  # Compute row index
    col = i % 6   # Compute column index
    mar_values[row][col] = frame[1]

# Print 2D array
print(mar_values)


[[0.50960974 0.50960974 0.52214837 0.52214837 0.62285294 0.62285294]
 [0.71745273 0.71745273 0.73982191 0.73982191 0.73982191 0.74989038]
 [0.55042813 0.55042813 0.55042813 0.58631947 0.58631947 0.58631947]
 [0.52496366 0.55860436 0.55860436 0.55860436 0.59156255 0.59156255]
 [0.54184348 0.56061395 0.56061395 0.56061395 0.56061395 0.56061395]
 [0.48218627 0.48218627 0.48218627 0.48873084 0.48873084 0.48873084]
 [0.58118424 0.57679727 0.57679727 0.57679727 0.57679727 0.66586027]]


In [50]:
# Mismatch Calculation

MOUTH_AR_THRESH = 0.485

error_rate = mismatch.calculate_mismatch_error_rate(mar_values, MOUTH_AR_THRESH)
print(error_rate)

14.285714285714285
